In [10]:
library("ggpubr")
library("cowplot")
library("tidyverse")
library(parallel)
library(pheatmap)
library(lme4)
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(stringr))

In [11]:
fileDescToDataframe_beta <- function(descriptors,
                                keep_labs = FALSE) {
  descriptors<-descriptors[[1]]
  assertthat::assert_that(typeof(descriptors) == "list")
  assertthat::assert_that(typeof(keep_labs) == "logical")
  
  do.call(
    rbind,
    lapply(
      descriptors,
      function(desc) {
        desc <- unlist(desc)
        desc <- desc[!grepl("scheme", names(desc))]
        names(desc) <- sub("^descriptors.","",names(desc))
        desc <- desc[!grepl("^specimens", names(desc))]
        desc <- desc[!grepl("^lab", names(desc))]
        desc <- desc[!grepl("^emr", names(desc))]
        desc <- desc[!grepl("^survey", names(desc))]
        desc <- desc[!grepl("^revision", names(desc))]
        desc <- desc[!grepl("^file.userTags", names(desc))]
        desc <- as.list(desc)
        df <- as.data.frame(desc)
        df
      }
    )
  )
}

fileDescToDataframe <- function(descriptors,
                                keep_labs = FALSE) {
  assertthat::assert_that(typeof(descriptors) == "list")
  assertthat::assert_that(typeof(keep_labs) == "logical")
  do.call(
    rbind,
    lapply(
      descriptors,
      function(desc) {
        desc <- unlist(desc)
        desc <- desc[!grepl("scheme", names(desc))]
        names(desc) <- sub("^descriptors.","",names(desc))
        desc <- desc[!grepl("^specimens", names(desc))]
        desc <- desc[!grepl("^lab", names(desc))]
        desc <- desc[!grepl("^emr", names(desc))]
        desc <- desc[!grepl("^survey", names(desc))]
        desc <- desc[!grepl("^revision", names(desc))]
        desc <- desc[!grepl("^file.userTags", names(desc))]

        desc <- as.list(desc)
        df <- as.data.frame(desc)
        df
      }
    )
  )
}


In [12]:
fres_first_batch <- hise::readFiles(list('c142f67d-403b-457f-ae17-accc5089399f'))

trying to submit request as query id...

user did not submit a query ID..

Retrying query on different end point...

Querying and downloading data onto IDE...



ERROR: Error in if (out[[fidx]]$descriptors$file$id != fileIds[[fidx]]) {: argument is of length zero


In [14]:
fres1 <- hise::readFiles(list("b494cce8-1314-4f6e-9666-42f0c6e1c702",
                              "00753770-4803-4947-a290-e7469c410067", 
                              "59f5f656-085f-4d0b-a240-68a9eb15e68e", 
                              "1a528317-bbf3-45b7-82c9-529577cf0b15", 
                              "55d5d20a-506a-459e-8315-7d66276fb8f9"))

Querying and downloading data onto IDE...



In [15]:
fres1<-fileDescToDataframe(fres1)


In [19]:
first_batch<-read_excel('cache/c142f67d-403b-457f-ae17-accc5089399f/20201752_Skene_1820_NPX_2020-12-03.xlsx')

In [20]:
first_batch$batch<-'First_Batch'

In [21]:
batch1<-read.csv(fres1$file[1])
batch2<-read.csv(fres1$file[2])
batch3<-read.csv(fres1$file[3])
batch4<-read.csv(fres1$file[4])
batch5<-read.csv(fres1$file[5])

In [31]:
batch1$batch<-"Batch1_Bridged"
batch2$batch<-"Batch2_Bridged"
batch3$batch<-"Batch3_Bridged"
batch4$batch<-"Batch4_Bridged"
batch5$batch<-"Batch5_Bridged"

In [32]:
batch3$sample.sampleKitGuid<-batch3$SampleKitGuid

In [33]:
batch1$sample.sampleKitGuid<-batch1$SampleKitGuid

In [34]:
batch5$sample.sampleKitGuid<-batch5$SampleKitGuid

In [35]:
first_batch$sample.sampleKitGuid<-paste0("KT",str_sub(first_batch$SampleID,3,7))
first_batch$NPX_bridged<-first_batch$NPX

In [36]:
all<-rbind.fill(list(batch1,batch2,batch3,batch4,batch5,first_batch))

In [37]:
all<-all%>% filter(!(Panel%in% c('Cardiometabolic','Neurology','Oncology')&Assay %in% c('TNF',"IL6","CXCL8")))

In [38]:
Olink<-unique(all[c("sample.sampleKitGuid","NPX_bridged","Assay",'batch')])

In [39]:
write.csv(Olink,'Olink.csv')